<div class="text_cell_render border-box-sizing rendered_html">
<h4>Course Recommendation System using Udemy Dataset<a class="anchor-link" href="#Course-Recommendation-System-using-Udemy-Dataset">¶</a>
</h4>
<h4>Workflow<a class="anchor-link" href="#Workflow">¶</a>
</h4>
<ul>
<li>Dataset</li>
<li>Vectorized our dataset</li>
<li>Cosine Similarity Matrix</li>
<li>ID,Score</li>
<li>Recommend</li>
</ul>

</div>

In [12]:
import pandas as pd
import numpy as np
import neattext.functions as nfx

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('udemy_course_data.csv')
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   course_id            3683 non-null   int64 
 1   course_title         3683 non-null   object
 2   url                  3683 non-null   object
 3   is_paid              3683 non-null   bool  
 4   price                3683 non-null   int64 
 5   num_subscribers      3683 non-null   int64 
 6   num_reviews          3683 non-null   int64 
 7   num_lectures         3683 non-null   int64 
 8   level                3683 non-null   object
 9   content_duration     3683 non-null   object
 10  published_timestamp  3683 non-null   object
 11  subject              3683 non-null   object
 12  profit               3683 non-null   int64 
 13  published_date       3683 non-null   object
 14  published_time       3682 non-null   object
 15  year                 3683 non-null   int64 
 16  month 

In [7]:
df['course_title'].iloc[1:5]

1    Complete GST Course & Certification - Grow You...
2    Financial Modeling for Business Analysts and C...
3    Beginner to Pro - Financial Analysis in Excel ...
4         How To Maximize Your Profits Trading Options
Name: course_title, dtype: object

In [8]:
# generating clean text by removing the stopwords and special characters

df['Clean_title'] = df['course_title'].apply(nfx.remove_stopwords)
df['Clean_title'] = df['Clean_title'].apply(nfx.remove_special_characters)
df['Clean_title'].iloc[1:5]

1    Complete GST Course  Certification  Grow Practice
2     Financial Modeling Business Analysts Consultants
3          Beginner Pro  Financial Analysis Excel 2017
4                     Maximize Profits Trading Options
Name: Clean_title, dtype: object

In [9]:
# vectorizing the course_title

countvect = CountVectorizer()
cv_mat = countvect.fit_transform(df['Clean_title'])
cv_mat

<3683x3564 sparse matrix of type '<class 'numpy.int64'>'
	with 18364 stored elements in Compressed Sparse Row format>

In [10]:
cv_mat.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
df_cv_words = pd.DataFrame(cv_mat.todense(),columns=countvect.get_feature_names())
df_cv_words

,000005,001,01,02,10,100,101,101master,102,10k,...,zend,zero,zerotohero,zf2,zinsen,zoho,zombie,zu,zuhause,zur
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3679,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# cosine similarity matrix

cosine_sim_mat = cosine_similarity(cv_mat)
cosine_sim_mat

array([[1.        , 0.20412415, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20412415, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.23570226],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.23570226, 0.        ,
        1.        ]])

In [15]:
# drop duplicates 

course_index = pd.Series(df.index,index = df['course_title']).drop_duplicates()

In [16]:
course_index

course_title
Ultimate Investment Banking Course                                0
Complete GST Course & Certification - Grow Your CA Practice       1
Financial Modeling for Business Analysts and Consultants          2
Beginner to Pro - Financial Analysis in Excel 2017                3
How To Maximize Your Profits Trading Options                      4
                                                               ... 
Learn jQuery from Scratch - Master of JavaScript library       3678
How To Design A WordPress Website With No Coding At All        3679
Learn and Build using Polymer                                  3680
CSS Animations: Create Amazing Effects on Your Website         3681
Using MODX CMS to Build Websites: A Beginner's Guide           3682
Length: 3683, dtype: int64

In [17]:
course_dict = dict(course_index)
course_keys = list(course_dict.keys())

touse = {}
counter = 0
keyword = 'Java'
for key,value in course_dict.items():
    if keyword in key:
        touse[counter] = key
        
    counter+=1
    
touse

{2473: 'Learning Dynamic Website Design - PHP MySQL and JavaScript',
 2484: 'Learn JavaScript for beginners',
 2498: 'Accelerated JavaScript Training',
 2506: 'Advanced Javascript',
 2515: 'Start Writing JavaScript Today - Beginner JavaScript Course',
 2520: 'Javascript ES6! A Complete Reference Guide to Javascript ES6',
 2526: 'Javascript Intermediate level 2 - Mastering Functions',
 2530: 'Learn JavaScript for Web Development',
 2536: 'Javascript Intermediate level 3 - JSON',
 2539: 'JavaScript DOM Dynamic Web interactive content Boot Camp',
 2540: 'Master JavaScript Programming, 3 Projects Included !!',
 2548: 'Dynamic JavaScript Master Class AJAX JSON Simple APIs',
 2551: 'JavaScript the Basics - JavaScript for Beginners',
 2552: 'Complete E-Commerce Course - Java,Spring,Hibernate and MySQL',
 2553: 'Learn SVG Animation - With HTML, CSS & Javascript',
 2580: 'JavaScript For Absolute Beginners - Level One',
 2593: 'Master ExpressJS to Build Web Apps with NodeJS&JavaScript',
 2612: '

In [18]:
temp = df[df['course_title'].str.contains('Python')]
temp.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,Clean_title
14,1196544,Python Algo Trading: Sentiment Trading with News,https://www.udemy.com/hedge-fund-strategy-trad...,True,200,294,19,42,All Levels,7 hours,2017-04-28T16:41:44Z,Business Finance,58800,2017-04-28,16:41:44Z,2017,4,28,Python Algo Trading Sentiment Trading News
30,1170894,Python Algo Stock Trading: Automate Your Trading!,https://www.udemy.com/algorithmic-stock-tradin...,True,95,1165,21,41,Beginner Level,2.5 hours,2017-05-28T23:41:03Z,Business Finance,110675,2017-05-28,23:41:03Z,2017,5,28,Python Algo Stock Trading Automate Trading
41,1035472,Python for Finance: Investment Fundamentals & ...,https://www.udemy.com/python-for-finance-inves...,True,195,3811,278,103,All Levels,6.5 hours,2017-03-30T22:17:09Z,Business Finance,743145,2017-03-30,22:17:09Z,2017,3,30,Python Finance Investment Fundamentals Data A...
149,1070886,Python Algo Trading: FX Trading with Oanda,https://www.udemy.com/python-algo-trading-fx-t...,True,200,453,42,33,Intermediate Level,3 hours,2017-03-14T00:39:45Z,Business Finance,90600,2017-03-14,00:39:45Z,2017,3,14,Python Algo Trading FX Trading Oanda
336,815482,Stock Technical Analysis with Python,https://www.udemy.com/stock-technical-analysis...,True,50,409,35,46,All Levels,8 hours,2016-04-12T00:40:03Z,Business Finance,20450,2016-04-12,00:40:03Z,2016,4,12,Stock Technical Analysis Python


In [19]:
top6 = temp.sort_values(by = 'num_subscribers',ascending=False).head(6)
top6

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,Clean_title
2502,16646,Web Programming with Python,https://www.udemy.com/web-programming-with-pyt...,True,50,35267,217,53,All Levels,4 hours,2012-04-25T00:01:43Z,Web Development,1763350,2012-04-25,00:01:43Z,2012,4,25,Web Programming Python
2575,47963,"Coding for Entrepreneurs: Learn Python, Django...",https://www.udemy.com/coding-for-entrepreneurs/,True,195,23412,799,251,All Levels,45 hours,2013-04-08T00:46:14Z,Web Development,4565340,2013-04-08,00:46:14Z,2013,4,8,Coding Entrepreneurs Learn Python Django More
2533,391546,Learn Python and Django: Payment Processing,https://www.udemy.com/learn-django-code-accept...,True,70,17714,198,23,All Levels,3.5 hours,2015-02-09T15:37:56Z,Web Development,1239980,2015-02-09,15:37:56Z,2015,2,9,Learn Python Django Payment Processing
3205,822444,Python and Django Full Stack Web Developer Boo...,https://www.udemy.com/python-and-django-full-s...,True,200,11832,1883,191,All Levels,31.5 hours,2017-02-24T18:40:55Z,Web Development,2366400,2017-02-24,18:40:55Z,2017,2,24,Python Django Stack Web Developer Bootcamp
3512,394832,Fun and creative web engineering with Python a...,https://www.udemy.com/web-engineering-with-pyt...,False,0,10917,319,25,All Levels,2 hours,2015-06-09T19:51:50Z,Web Development,0,2015-06-09,19:51:50Z,2015,6,9,Fun creative web engineering Python Web2py
2558,938560,The Complete Ethical Hacking Course 2.0: Pytho...,https://www.udemy.com/penetration-testing-ethi...,True,195,7827,268,66,All Levels,11 hours,2016-09-26T15:08:29Z,Web Development,1526265,2016-09-26,15:08:29Z,2016,9,26,Complete Ethical Hacking Course 20 Python Kal...


In [20]:
index = course_index['How To Maximize Your Profits Trading Options']

In [22]:
scores = list(enumerate(cosine_sim_mat[index]))
scores

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 1.0),
 (5, 0.20412414523193154),
 (6, 0.20412414523193154),
 (7, 0.1889822365046136),
 (8, 0.3779644730092272),
 (9, 0.0),
 (10, 0.20412414523193154),
 (11, 0.5),
 (12, 0.0),
 (13, 0.17677669529663687),
 (14, 0.35355339059327373),
 (15, 0.0),
 (16, 0.0),
 (17, 0.1889822365046136),
 (18, 0.22360679774997896),
 (19, 0.0),
 (20, 0.30151134457776363),
 (21, 0.20412414523193154),
 (22, 0.1889822365046136),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.1889822365046136),
 (30, 0.35355339059327373),
 (31, 0.0),
 (32, 0.0),
 (33, 0.3779644730092272),
 (34, 0.0),
 (35, 0.3779644730092272),
 (36, 0.30151134457776363),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.5669467095138407),
 (44, 0.35355339059327373),
 (45, 0.0),
 (46, 0.35355339059327373),
 (47, 0.0),
 (48, 0.35355339059327373),
 (49, 0.4082482904638631),
 (50, 0.22360679774997896),
 (51, 0.0),
 (52, 0.1889822365046136),


In [23]:
sorted_score = sorted(scores,key = lambda x:x[1],reverse=True)
sorted_score

[(4, 1.0),
 (410, 0.5773502691896258),
 (43, 0.5669467095138407),
 (96, 0.5303300858899106),
 (138, 0.5303300858899106),
 (195, 0.5303300858899106),
 (444, 0.5303300858899106),
 (803, 0.5303300858899106),
 (11, 0.5),
 (59, 0.5),
 (68, 0.5),
 (71, 0.5),
 (97, 0.5),
 (330, 0.5),
 (378, 0.5),
 (514, 0.5),
 (647, 0.5),
 (738, 0.5),
 (947, 0.5),
 (991, 0.5),
 (811, 0.45226701686664544),
 (66, 0.4472135954999579),
 (222, 0.4472135954999579),
 (234, 0.4472135954999579),
 (369, 0.4472135954999579),
 (439, 0.4472135954999579),
 (463, 0.4472135954999579),
 (766, 0.4472135954999579),
 (829, 0.4472135954999579),
 (399, 0.4330127018922194),
 (49, 0.4082482904638631),
 (54, 0.4082482904638631),
 (72, 0.4082482904638631),
 (85, 0.4082482904638631),
 (107, 0.4082482904638631),
 (160, 0.4082482904638631),
 (167, 0.4082482904638631),
 (205, 0.4082482904638631),
 (295, 0.4082482904638631),
 (353, 0.4082482904638631),
 (402, 0.4082482904638631),
 (411, 0.4082482904638631),
 (802, 0.4082482904638631),
 (95

In [24]:
# so the sorted score list is a collection of tuples 
# which have the index and the value,so i will select the indices first

sorted_indices = [i[0] for i in sorted_score[1:]]
sorted_values = [i[1] for i in sorted_score[1:]]
sorted_values

[0.5773502691896258,
 0.5669467095138407,
 0.5303300858899106,
 0.5303300858899106,
 0.5303300858899106,
 0.5303300858899106,
 0.5303300858899106,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.45226701686664544,
 0.4472135954999579,
 0.4472135954999579,
 0.4472135954999579,
 0.4472135954999579,
 0.4472135954999579,
 0.4472135954999579,
 0.4472135954999579,
 0.4472135954999579,
 0.4330127018922194,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.4082482904638631,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.3779644730092272,
 0.

In [25]:
recommended_result_df = df.iloc[sorted_indices]

recommended_result_df

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,Clean_title
410,889066,Trading Options Basics,https://www.udemy.com/trading-options-basics/,True,200,8,0,8,Beginner Level,1.5 hours,2016-07-01T03:13:22Z,Business Finance,1600,2016-07-01,03:13:22Z,2016,7,1,Trading Options Basics
43,627540,Options Trading - How to Win with Weekly Options,https://www.udemy.com/work-from-home-setup-you...,True,115,7489,1190,25,Intermediate Level,1 hour,2015-10-22T21:54:28Z,Business Finance,861235,2015-10-22,21:54:28Z,2015,10,22,Options Trading Win Weekly Options
96,474928,Intermediate Options trading concepts for Stoc...,https://www.udemy.com/intermediate-options-tra...,True,40,2000,30,9,All Levels,1 hour,2015-04-13T20:28:04Z,Business Finance,80000,2015-04-13,20:28:04Z,2015,4,13,Intermediate Options trading concepts Stocks O...
138,1243448,Forex Trading with Fixed 'Risk through Options...,https://www.udemy.com/forexoptions/,True,200,611,4,26,Beginner Level,1 hour,2017-06-07T17:15:07Z,Business Finance,122200,2017-06-07,17:15:07Z,2017,6,7,Forex Trading Fixed Risk Options Trading
195,919906,Trading Options For Consistent Returns: Option...,https://www.udemy.com/trading-options-for-income/,False,0,4077,281,20,Beginner Level,1.5 hours,2016-08-18T21:57:04Z,Business Finance,0,2016-08-18,21:57:04Z,2016,8,18,Trading Options Consistent Returns Options Basics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,775618,Learn jQuery from Scratch - Master of JavaScri...,https://www.udemy.com/easy-jquery-for-beginner...,True,100,1040,14,21,All Levels,2 hours,2016-06-14T17:36:46Z,Web Development,104000,2016-06-14,17:36:46Z,2016,6,14,Learn jQuery Scratch Master JavaScript library
3679,1088178,How To Design A WordPress Website With No Codi...,https://www.udemy.com/how-to-make-a-wordpress-...,True,25,306,3,42,Beginner Level,3.5 hours,2017-03-10T22:24:30Z,Web Development,7650,2017-03-10,22:24:30Z,2017,3,10,Design WordPress Website Coding
3680,635248,Learn and Build using Polymer,https://www.udemy.com/learn-and-build-using-po...,True,40,513,169,48,All Levels,3.5 hours,2015-12-30T16:41:42Z,Web Development,20520,2015-12-30,16:41:42Z,2015,12,30,Learn Build Polymer
3681,905096,CSS Animations: Create Amazing Effects on Your...,https://www.udemy.com/css-animations-create-am...,True,50,300,31,38,All Levels,3 hours,2016-08-11T19:06:15Z,Web Development,15000,2016-08-11,19:06:15Z,2016,8,11,CSS Animations Create Amazing Effects Website


In [26]:
recommended_result_df['Similarity_Score'] = np.array(sorted_values)
recommended_result_df

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,Clean_title,Similarity_Score
410,889066,Trading Options Basics,https://www.udemy.com/trading-options-basics/,True,200,8,0,8,Beginner Level,1.5 hours,2016-07-01T03:13:22Z,Business Finance,1600,2016-07-01,03:13:22Z,2016,7,1,Trading Options Basics,0.577350
43,627540,Options Trading - How to Win with Weekly Options,https://www.udemy.com/work-from-home-setup-you...,True,115,7489,1190,25,Intermediate Level,1 hour,2015-10-22T21:54:28Z,Business Finance,861235,2015-10-22,21:54:28Z,2015,10,22,Options Trading Win Weekly Options,0.566947
96,474928,Intermediate Options trading concepts for Stoc...,https://www.udemy.com/intermediate-options-tra...,True,40,2000,30,9,All Levels,1 hour,2015-04-13T20:28:04Z,Business Finance,80000,2015-04-13,20:28:04Z,2015,4,13,Intermediate Options trading concepts Stocks O...,0.530330
138,1243448,Forex Trading with Fixed 'Risk through Options...,https://www.udemy.com/forexoptions/,True,200,611,4,26,Beginner Level,1 hour,2017-06-07T17:15:07Z,Business Finance,122200,2017-06-07,17:15:07Z,2017,6,7,Forex Trading Fixed Risk Options Trading,0.530330
195,919906,Trading Options For Consistent Returns: Option...,https://www.udemy.com/trading-options-for-income/,False,0,4077,281,20,Beginner Level,1.5 hours,2016-08-18T21:57:04Z,Business Finance,0,2016-08-18,21:57:04Z,2016,8,18,Trading Options Consistent Returns Options Basics,0.530330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,775618,Learn jQuery from Scratch - Master of JavaScri...,https://www.udemy.com/easy-jquery-for-beginner...,True,100,1040,14,21,All Levels,2 hours,2016-06-14T17:36:46Z,Web Development,104000,2016-06-14,17:36:46Z,2016,6,14,Learn jQuery Scratch Master JavaScript library,0.000000
3679,1088178,How To Design A WordPress Website With No Codi...,https://www.udemy.com/how-to-make-a-wordpress-...,True,25,306,3,42,Beginner Level,3.5 hours,2017-03-10T22:24:30Z,Web Development,7650,2017-03-10,22:24:30Z,2017,3,10,Design WordPress Website Coding,0.000000
3680,635248,Learn and Build using Polymer,https://www.udemy.com/learn-and-build-using-po...,True,40,513,169,48,All Levels,3.5 hours,2015-12-30T16:41:42Z,Web Development,20520,2015-12-30,16:41:42Z,2015,12,30,Learn Build Polymer,0.000000
3681,905096,CSS Animations: Create Amazing Effects on Your...,https://www.udemy.com/css-animations-create-am...,True,50,300,31,38,All Levels,3 hours,2016-08-11T19:06:15Z,Web Development,15000,2016-08-11,19:06:15Z,2016,8,11,CSS Animations Create Amazing Effects Website,0.000000


In [27]:
use_df = recommended_result_df[['Clean_title','Similarity_Score']]
use_df

,Clean_title,Similarity_Score
410,Trading Options Basics,0.577350
43,Options Trading Win Weekly Options,0.566947
96,Intermediate Options trading concepts Stocks O...,0.530330
138,Forex Trading Fixed Risk Options Trading,0.530330
195,Trading Options Consistent Returns Options Basics,0.530330
...,...,...
3678,Learn jQuery Scratch Master JavaScript library,0.000000
3679,Design WordPress Website Coding,0.000000
3680,Learn Build Polymer,0.000000
3681,CSS Animations Create Amazing Effects Website,0.000000


In [36]:
def recommend_course(title, numrec=10):
    
    course_index = pd.Series(
        df.index, index=df['course_title']).drop_duplicates()

    index = course_index[title]
    scores = list(enumerate(cosine_sim_mat [index]))
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    selected_course_index = [i[0] for i in sorted_scores[1:]]
    selected_course_score = [i[1] for i in sorted_scores[1:]]
    rec_df = df.iloc[selected_course_index]
    rec_df['Similarity_Score'] = selected_course_score
    final_recommended_courses = rec_df[[
        'course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']]
    return final_recommended_courses.head(numrec)

In [45]:
ans = recommend_course('Web Programming with Python', numrec=10)
ans

,course_title,Similarity_Score,url,price,num_subscribers
3348,Python Web Programming,1.000000,https://www.udemy.com/python-web-programming/,100,1020
3143,Complete Python Web Course: Build 8 Python Web...,0.666667,https://www.udemy.com/the-complete-python-web-...,110,7489
2686,Python for Beginners: Python Programming Langu...,0.612372,https://www.udemy.com/python-course/,150,6153
3338,Introduction to QGIS Python Programming,0.577350,https://www.udemy.com/introduction-to-qgis-pyt...,85,197
2712,Back to School Web Development and Programming...,0.516398,https://www.udemy.com/back-to-school-web-devel...,200,18170
2864,Introduction to web programming for GIS applic...,0.516398,https://www.udemy.com/introduction-to-web-prog...,100,205
3165,Professional Python Web Development Using Flask,0.516398,https://www.udemy.com/python-flask-course/,120,3420
3244,Web Programming Course : Asp.Net MVC,0.516398,https://www.udemy.com/mvc-course/,100,2760
2512,Web Development w/ Google’s Go (golang) Progra...,0.471405,https://www.udemy.com/go-programming-language/,65,4872
2857,The Complete TypeScript Programming Guide for ...,0.471405,https://www.udemy.com/typescript-programming/,40,236
